In [1]:
# !sudo apt install swig

In [2]:
# !pip install quantstats
# !pip install scikit-learn
# !pip install torch
# !pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

# Initializing

In [1]:
import logging
logging.getLogger('matplotlib.font_manager').disabled = True
import torch

import numpy as np
import pandas as pd

from sklearn.preprocessing import MaxAbsScaler
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import GroupByScaler
from utils import PortfolioOptimizationEnv, custom_reward_function, sharpe_ratio_reward_function
from finrl.agents.portfolio_optimization.models import DRLAgent
from finrl.agents.portfolio_optimization.architectures import EIIE

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Data Import and Preprocessing

In [2]:
TEST_SET = [
    "AAPL", "CVX", "GS", "JNJ",
    "JPM", "MSFT", "PFE", "PG",
    "GOOG", "XOM"
]
START_DATE = '2011-01-01'
END_DATE = '2023-12-31'
print(len(TEST_SET))

##COVARIATE 1: PRICES

portfolio_raw_df = YahooDownloader(start_date = START_DATE,
                                end_date = END_DATE,
                                ticker_list = TEST_SET).fetch_data()

10


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (32700, 8)


In [3]:
##COVARIATES 2,3: VIX, FEAR/GREED INDEX

combined_spy_vix_fear_greed = pd.read_csv('datasets/VIX_feargreed/spy_vix_fear_greed_2011_2023.csv')


portfolio_raw_df['date'] = pd.to_datetime(portfolio_raw_df['date'])
combined_spy_vix_fear_greed['Date'] = pd.to_datetime(combined_spy_vix_fear_greed['Date'])


df_portfolio_comb = pd.merge(portfolio_raw_df, combined_spy_vix_fear_greed, left_on='date', right_on='Date', how='left')


df_portfolio_comb = df_portfolio_comb.drop(columns=['Date'])
df_portfolio_comb['date'] = df_portfolio_comb['date'].dt.strftime('%Y-%m-%d')

In [4]:
##COVARIATE 4: STOCK HISTORICAL RATINGS

FMP_historical_ratings = pd.read_csv('datasets/historical_ratings/FMP_historical_ratings.csv')

df_portfolio_comb_2 = df_portfolio_comb.merge(
    FMP_historical_ratings[['date', 'symbol', 'ratingScore', 'ratingDetailsDCFScore', 'ratingDetailsROEScore', 'ratingDetailsROAScore', 'ratingDetailsPEScore', 'ratingDetailsPBScore']],
    left_on=['date', 'tic'],
    right_on=['date', 'symbol'],
    how='left' )

df_portfolio_comb_2.drop(columns=['symbol'], inplace=True)

In [5]:
##COVARIATE 5: Macro Indicators

expenses = pd.read_csv('datasets/macro_indicators/expenses.csv')
gdp = pd.read_csv('datasets/macro_indicators/gdp.csv')
goods_services = pd.read_csv('datasets/macro_indicators/goods_and_services.csv')
pound_dollar = pd.read_csv('datasets/macro_indicators/pound-dollar-exchange-rate-historical-chart.csv')
unemployment = pd.read_csv('datasets/macro_indicators/unemployment.csv')
index_consumer_services = pd.read_csv('datasets/macro_indicators/index_consumer_services.csv')

df_portfolio_comb_2['date'] = pd.to_datetime(df_portfolio_comb_2['date'])
expenses['date'] = pd.to_datetime(unemployment['date'])
gdp['date'] = pd.to_datetime(gdp['DATE'])
goods_services['date'] = pd.to_datetime(goods_services['date'])
pound_dollar['date'] = pd.to_datetime(pound_dollar['date'])
unemployment['date'] = pd.to_datetime(unemployment['date'])
index_consumer_services['date'] = pd.to_datetime(index_consumer_services['date'])

df_portfolio_comb_3 = pd.merge(df_portfolio_comb_2, expenses, on='date', how='left')
df_portfolio_comb_3 = pd.merge(df_portfolio_comb_3, gdp, on='date', how='left')
df_portfolio_comb_3 = pd.merge(df_portfolio_comb_3, goods_services, on='date', how='left')
df_portfolio_comb_3 = pd.merge(df_portfolio_comb_3, pound_dollar, on='date', how='left')
df_portfolio_comb_3 = pd.merge(df_portfolio_comb_3, unemployment, on='date', how='left')
df_portfolio_comb_3 = pd.merge(df_portfolio_comb_3, index_consumer_services, on='date', how='left')

# Sort the dataframe by date (and optionally 'tic' if needed)
df_portfolio_comb_3 = df_portfolio_comb_3.sort_values(by=['date', 'tic'])

# Forward fill missing data
df_portfolio_comb_3['expenses'] = df_portfolio_comb_3['expenses'].ffill()
df_portfolio_comb_3['GDP'] = df_portfolio_comb_3['GDP'].ffill()
df_portfolio_comb_3['exports'] = df_portfolio_comb_3['exports'].ffill()
df_portfolio_comb_3['imports'] = df_portfolio_comb_3['imports'].ffill()
df_portfolio_comb_3['pound_dollar_exchange_rate'] = df_portfolio_comb_3['pound_dollar_exchange_rate'].ffill()
df_portfolio_comb_3['unemployment'] = df_portfolio_comb_3['unemployment'].ffill()
df_portfolio_comb_3['ics'] = df_portfolio_comb_3['ics'].ffill()
df_portfolio_comb_3.drop(columns=['DATE'], inplace=True)

In [6]:
##COVARIATE 6: Fundamental Indicators

DBITDA_ratio = pd.read_csv('datasets/fundemental indicators/DBITDA_ratio.csv')
PB_ratio = pd.read_csv('datasets/fundemental indicators/PB_ratio.csv')
PE_ratio = pd.read_csv('datasets/fundemental indicators/PE_ratio.csv')
PS_ratio = pd.read_csv('datasets/fundemental indicators/PS_ratio.csv')
quarterly_data = pd.read_csv('datasets/fundemental indicators/quaterlydata.csv')
ROE = pd.read_csv('datasets/fundemental indicators/ROE.csv')

df_portfolio_comb_3['date'] = pd.to_datetime(df_portfolio_comb_3['date'])
DBITDA_ratio['date'] = pd.to_datetime(DBITDA_ratio['date'])
PB_ratio['date'] = pd.to_datetime(PB_ratio['date'])
PE_ratio['date'] = pd.to_datetime(PE_ratio['date'])
PS_ratio['date'] = pd.to_datetime(PS_ratio['date'])
quarterly_data['date'] = pd.to_datetime(quarterly_data['date'])
ROE['date'] = pd.to_datetime(ROE['date'])

df_portfolio_comb_4 = df_portfolio_comb_3

df_portfolio_comb_4 = pd.merge(df_portfolio_comb_4, DBITDA_ratio, left_on=['tic', 'date'], right_on=['ticker', 'date'], how='left')
df_portfolio_comb_4 = df_portfolio_comb_4.drop(columns=['ticker'])
df_portfolio_comb_4 = pd.merge(df_portfolio_comb_4, PB_ratio, left_on=['tic', 'date'], right_on=['ticker', 'date'], how='left')
df_portfolio_comb_4 = df_portfolio_comb_4.drop(columns=['ticker','Stock Price'])
df_portfolio_comb_4 = pd.merge(df_portfolio_comb_4, PE_ratio, left_on=['tic', 'date'], right_on=['ticker', 'date'], how='left')
df_portfolio_comb_4 = df_portfolio_comb_4.drop(columns=['ticker','Stock Price'])
df_portfolio_comb_4 = pd.merge(df_portfolio_comb_4, PS_ratio, left_on=['tic', 'date'], right_on=['ticker', 'date'], how='left')
df_portfolio_comb_4 = df_portfolio_comb_4.drop(columns=['ticker','Stock Price'])
df_portfolio_comb_4 = pd.merge(df_portfolio_comb_4, quarterly_data, left_on=['tic', 'date'], right_on=['ticker', 'date'], how='left')
df_portfolio_comb_4 = df_portfolio_comb_4.drop(columns=['ticker'])
df_portfolio_comb_4 = pd.merge(df_portfolio_comb_4, ROE, left_on=['tic', 'date'], right_on=['ticker', 'date'], how='left')
df_portfolio_comb_4 = df_portfolio_comb_4.drop(columns=['ticker'])

# Sort the dataframe by date (and optionally 'tic' if needed)
df_portfolio_comb_4 = df_portfolio_comb_4.sort_values(by=['tic', 'date'])

# Forward fill missing data
df_portfolio_comb_4['TTM Revenue(Billion)'] = df_portfolio_comb_4['TTM Revenue(Billion)'].ffill()
df_portfolio_comb_4['TTM EBITDA(Billion)'] = df_portfolio_comb_4['TTM EBITDA(Billion)'].ffill()
df_portfolio_comb_4['EBITDA Margin'] = df_portfolio_comb_4['EBITDA Margin'].ffill()
df_portfolio_comb_4['Price to Book Ratio'] = df_portfolio_comb_4['Price to Book Ratio'].ffill()
df_portfolio_comb_4['PE Ratio'] = df_portfolio_comb_4['PE Ratio'].ffill()
df_portfolio_comb_4['Price to Sales Ratio'] = df_portfolio_comb_4['Price to Sales Ratio'].ffill()
df_portfolio_comb_4['Assets'] = df_portfolio_comb_4['Assets'].ffill()
df_portfolio_comb_4['NetIncomeLoss'] = df_portfolio_comb_4['NetIncomeLoss'].ffill()
df_portfolio_comb_4['Return on Equity(%)'] = df_portfolio_comb_4['Return on Equity(%)'].ffill()

C:\Users\limyi\AppData\Local\Temp\ipykernel_15872\1570567354.py:11: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  DBITDA_ratio['date'] = pd.to_datetime(DBITDA_ratio['date'])
C:\Users\limyi\AppData\Local\Temp\ipykernel_15872\1570567354.py:12: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  PB_ratio['date'] = pd.to_datetime(PB_ratio['date'])
C:\Users\limyi\AppData\Local\Temp\ipykernel_15872\1570567354.py:13: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  PE_ratio['date'] = pd.to_datetime(PE_ratio['date'])
C:\Users\limyi\AppData\Local\Temp\ipykernel_15872\1570567354.py:14: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the defa

In [7]:
df_portfolio_final = df_portfolio_comb_4

In [8]:
# Check the data types of all columns in the DataFrame
print(df_portfolio_final.dtypes)

date                              datetime64[ns]
open                                     float64
high                                     float64
low                                      float64
close                                    float64
volume                                     int64
tic                                       object
day                                        int32
Open_VIX                                 float64
High_VIX                                 float64
Low_VIX                                  float64
Close_VIX                                float64
Adj Close_VIX                            float64
Open_SPY                                 float64
High_SPY                                 float64
Low_SPY                                  float64
Close_SPY                                float64
Adj Close_SPY                            float64
Volume_SPY                                 int64
Fear Greed                               float64
ratingScore         

In [9]:
#Convert Str to Float64
df_portfolio_final['net_export_goods_and_services'] = df_portfolio_final['net_export_goods_and_services'].str.replace(',', '').astype('float64')
df_portfolio_final['expenses'] = df_portfolio_final['expenses'].str.replace(',', '').astype('float64')
df_portfolio_final['exports'] = df_portfolio_final['exports'].str.replace(',', '').astype('float64')
df_portfolio_final['exports_goods'] = df_portfolio_final['exports_goods'].str.replace(',', '').astype('float64')
df_portfolio_final['exports_services'] = df_portfolio_final['exports_services'].str.replace(',', '').astype('float64')
df_portfolio_final['imports'] = df_portfolio_final['imports'].str.replace(',', '').astype('float64')
df_portfolio_final['imports_goods'] = df_portfolio_final['imports_goods'].str.replace(',', '').astype('float64')
df_portfolio_final['imports_services'] = df_portfolio_final['imports_services'].str.replace(',', '').astype('float64')

#Convert rest to Float64
df_portfolio_final[df_portfolio_final.columns.difference(['date', 'tic'])] = df_portfolio_final[df_portfolio_final.columns.difference(['date', 'tic'])].astype('float64')

#Convert date back
df_portfolio_final['date'] = df_portfolio_final['date'].dt.strftime('%Y-%m-%d')

In [10]:
# Filling NaN values with 0
df_portfolio_final.fillna(0, inplace=True)

In [11]:
portfolio_norm_df = GroupByScaler(by="tic", scaler=MaxAbsScaler).fit_transform(df_portfolio_final)
portfolio_norm_df

,date,open,high,low,close,volume,tic,day,Open_VIX,High_VIX,...,Price to Book Ratio,Net EPS,PE Ratio,TTM Sales per Share,Price to Sales Ratio,Assets,NetIncomeLoss,TTM Net Income (Billions),Shareholder's Equity (Billion),Return on Equity(%)
0,2011-01-03,0.058731,0.059087,0.058891,0.050363,0.236650,AAPL,0.00,0.216955,0.210015,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
10,2011-01-04,0.059958,0.059488,0.059491,0.050626,0.164317,AAPL,0.25,0.209699,0.213408,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
20,2011-01-05,0.059437,0.059817,0.059735,0.051040,0.135843,AAPL,0.50,0.215383,0.210015,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
30,2011-01-06,0.060369,0.059980,0.060352,0.050999,0.159718,AAPL,0.75,0.203168,0.205452,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
40,2011-01-07,0.060237,0.060177,0.060170,0.051364,0.165833,AAPL,1.00,0.209336,0.211419,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32659,2023-12-22,0.852725,0.852858,0.861494,0.855590,0.152474,XOM,1.00,0.165921,0.163332,...,0.269648,0.0,0.223856,0.0,0.253363,0.963788,0.561798,0.0,0.0,0.807644
32669,2023-12-26,0.856309,0.853604,0.864032,0.857521,0.198649,XOM,0.25,0.166526,0.161460,...,0.269648,0.0,0.223856,0.0,0.253363,0.963788,0.561798,0.0,0.0,0.807644
32679,2023-12-27,0.850475,0.849627,0.857433,0.853491,0.171790,XOM,0.50,0.157456,0.152568,...,0.269648,0.0,0.223856,0.0,0.253363,0.963788,0.561798,0.0,0.0,0.807644
32689,2023-12-28,0.845057,0.841839,0.847195,0.841150,0.192681,XOM,0.75,0.150441,0.148005,...,0.269648,0.0,0.223856,0.0,0.253363,0.963788,0.561798,0.0,0.0,0.807644


In [12]:
df_portfolio = portfolio_norm_df[["date", 
                                  "tic", 
                                  "close",
                                  "high", 
                                  "low",
                                  'High_VIX',
                                  'Low_VIX',
                                  'Close_VIX',
                                  'High_SPY',
                                  'Low_SPY',
                                  'Close_SPY',
                                  'Volume_SPY',
                                  'Fear Greed', 
                                  'ratingScore',
                                  'ratingDetailsDCFScore',
                                  'ratingDetailsROEScore',
                                  'ratingDetailsROAScore',
                                  'ratingDetailsPEScore',
                                  'ratingDetailsPBScore',
                                  'expenses',
                                  'GDP', 
                                  'exports', 
                                  'imports',
                                  'pound_dollar_exchange_rate', 
                                  'unemployment',
                                  'ics',
                                  'TTM Revenue(Billion)',
                                  'TTM EBITDA(Billion)',
                                  'EBITDA Margin',
                                  'Price to Book Ratio',
                                  'PE Ratio',
                                  'Price to Sales Ratio',
                                  'Assets',
                                  'NetIncomeLoss',
                                  'Return on Equity(%)']]

In [13]:
df_portfolio_train = df_portfolio[(df_portfolio["date"] >= START_DATE) & (df_portfolio["date"] < "2020-12-31")]
df_portfolio_test = df_portfolio[(df_portfolio["date"] >= "2021-01-01") & (df_portfolio["date"] < "2022-12-31")]

# Feature Selection and DRL Model Parameters

In [15]:
import pandas as pd
from itertools import product
import numpy as np  # for NaN
import torch

FEATURE_NAMES = ["close",
                  #"high", 
                  #"low",
                  #'High_VIX',
                  #'Low_VIX',
                  #'Close_VIX',
                  'High_SPY',
                  #'Low_SPY',
                  'Close_SPY',
                  #'Volume_SPY',
                  'Fear Greed', 
                  'ratingScore',
                  #'ratingDetailsDCFScore',
                  #'ratingDetailsROEScore',
                  #'ratingDetailsROAScore',
                  #'ratingDetailsPEScore',
                  #'ratingDetailsPBScore',
                  #'expenses',
                  #'GDP', 
                  #'exports', 
                  #'imports',
                  'pound_dollar_exchange_rate', 
                  'unemployment',
                  #'ics',
                  #'TTM Revenue(Billion)',
                  #'TTM EBITDA(Billion)',
                  'EBITDA Margin',
                  'Price to Book Ratio',
                  'PE Ratio',
                  #'Price to Sales Ratio',
                  #'Assets',
                  #'NetIncomeLoss',
                  'Return on Equity(%)']

COMISSION_FEE = 0.0025

# Define hyperparameter ranges
time_windows = [100,150,50]
k_sizes = [3,4,5]
conv_mids = [5,10,20]
conv_finals = [5,10,20]
lr = [0.01,0.02]

# Function to calculate the final accumulative portfolio value
def calculate_cumulative_value(portfolio_values):
    initial_value = portfolio_values[0]
    final_value = portfolio_values[-1]
    return final_value / initial_value

# Function to calculate the maximum drawdown
def calculate_max_drawdown(portfolio_values):
    portfolio_values = np.array(portfolio_values)
    running_max = np.maximum.accumulate(portfolio_values)
    drawdowns = (portfolio_values - running_max) / running_max
    max_drawdown = drawdowns.min()
    
    return max_drawdown

# Function to calculate the Sharpe ratio
def calculate_sharpe_ratio(portfolio_values, risk_free_rate=0):
    returns = np.diff(portfolio_values) / portfolio_values[:-1]
    excess_returns = returns - risk_free_rate
    sharpe_ratio = np.mean(excess_returns) / np.std(excess_returns)
    sharpe_ratio_annualized = sharpe_ratio * np.sqrt(252)
    
    return sharpe_ratio_annualized


# Define a function to calculate the metrics (this is just a placeholder; replace with actual functions)
def calculate_metrics(environment):
    final_portfolio_value = environment._asset_memory["final"][-1]
    max_drawdown = calculate_max_drawdown(environment._asset_memory["final"])
    sharpe_ratio = calculate_sharpe_ratio(environment._asset_memory["final"])
    cumulative_value = calculate_cumulative_value(environment._asset_memory["final"])
    return final_portfolio_value, max_drawdown, sharpe_ratio, cumulative_value

# DataFrame to store results
columns = ["TIME_WINDOW", "K_SIZE", "CONV_MID", "CONV_FINAL","LEARNING_RATE", 
        'train_portfolio_value', 'train_drawdown', 'train_sharpe', 'train_cumulative',
        'test_portfolio_value', 'test_drawdown', 'test_sharpe', 'test_cumulative']

results_df = pd.DataFrame(columns=columns)

In [16]:
# Hyperparameter tuning loop with 5 episodes of training and error handling
for TIME_WINDOW, K_SIZE, CONV_MID, CONV_FINAL,LEARNING_RATE in product(time_windows, k_sizes, conv_mids, conv_finals, lr):
    try:
        # Update the policy_kwargs for the current set of hyperparameters
        policy_kwargs = {
        "initial_features": len(FEATURE_NAMES),
        "k_size": K_SIZE,
        "time_window": TIME_WINDOW,
        "conv_mid_features":CONV_MID,
        "conv_final_features":CONV_FINAL}

        # Create the environment for training and the test years
        environment = PortfolioOptimizationEnv(df_portfolio_train, initial_amount=100000, comission_fee_pct=COMISSION_FEE,
                                               time_window=TIME_WINDOW, features=FEATURE_NAMES, normalize_df=None,
                                               reward_function=custom_reward_function, reward_scaling=1.0)

        environment_test = PortfolioOptimizationEnv(df_portfolio_test, initial_amount=100000, comission_fee_pct=COMISSION_FEE,
                                                    time_window=TIME_WINDOW, features=FEATURE_NAMES, normalize_df=None,
                                                    reward_function=custom_reward_function, reward_scaling=1.0)

        # Instantiate the model for each combination of hyperparameters
        # set PolicyGradient parameters
        model_kwargs = {
                "lr": LEARNING_RATE,
                "policy": EIIE,
                }

        EIIE_model = DRLAgent(environment).get_model("pg", device, model_kwargs, policy_kwargs)
        EIIE_policy = EIIE(time_window=TIME_WINDOW, device=device, initial_features=len(FEATURE_NAMES),
                           k_size=K_SIZE, conv_mid_features=CONV_MID, conv_final_features=CONV_FINAL)

        # Train the model for 10 episodes
        DRLAgent.train_model(EIIE_model, episodes=10)
        
        # Save the model policy after training
        model_filename = f"policy_EIIE_{TIME_WINDOW}_{K_SIZE}_{CONV_MID}_{CONV_FINAL}_{LEARNING_RATE}.pt"
        torch.save(EIIE_model.train_policy.state_dict(), model_filename)
        
        EIIE_policy = EIIE(time_window = TIME_WINDOW, device = device, initial_features = len(FEATURE_NAMES),
                          k_size = K_SIZE, conv_mid_features = CONV_MID,conv_final_features = CONV_FINAL)

        EIIE_policy.load_state_dict(torch.load(model_filename))

        # Training evaluation
        environment.reset()
        DRLAgent.DRL_validation(EIIE_model, environment, policy=EIIE_policy)
        train_final_value, train_drawdown, train_sharpe, train_cumulative = calculate_metrics(environment)

        # Test evaluation
        environment_test.reset()
        DRLAgent.DRL_validation(EIIE_model, environment_test, policy=EIIE_policy)
        test_final_value, test_drawdown, test_sharpe, test_cumulative = calculate_metrics(environment_test)

 

    except Exception as e:
        print(f"Error encountered with combination policy_EIIE_{TIME_WINDOW}_{K_SIZE}_{CONV_MID}_{CONV_FINAL}: {e}")
        # If an error occurs, store NaN for all the metrics
        train_final_value = train_drawdown = train_sharpe = train_cumulative = np.nan
        test_final_value = test_drawdown = test_sharpe = test_cumulative = np.nan
 

    # Create a temporary DataFrame to store the current results
    temp_df = pd.DataFrame([{
        "TIME_WINDOW": TIME_WINDOW, "K_SIZE": K_SIZE, "CONV_MID": CONV_MID, "CONV_FINAL": CONV_FINAL, "LEARNING_RATE": LEARNING_RATE,
        "train_portfolio_value": train_final_value, "train_drawdown": train_drawdown, "train_sharpe": train_sharpe, "train_cumulative": train_cumulative,
        "test_portfolio_value": test_final_value, "test_drawdown": test_drawdown, "test_sharpe": test_sharpe, "test_cumulative": test_cumulative,
        }])
    
    # Use pd.concat instead of append to add the results to the main DataFrame
    results_df = pd.concat([results_df, temp_df], ignore_index=True)
    
    try:
        results_df.to_csv('results_df.csv', index=False)
    except Exception as e:
        print(f"An error occurred: {e}")
    
results_df

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Initial portfolio value:100000
Final portfolio value: 407975.5625
Final accumulative portfolio value: 4.079755625
Maximum DrawDown: -0.3030059760208773
Sharpe ratio: 0.9028789482347024


 10%|████████▎                                                                          | 1/10 [00:22<03:21, 22.35s/it]

Initial portfolio value:100000
Final portfolio value: 949532.8125
Final accumulative portfolio value: 9.495328125
Maximum DrawDown: -0.3197392456832293
Sharpe ratio: 1.138621350439884


 20%|████████████████▌                                                                  | 2/10 [00:44<02:58, 22.34s/it]

...

 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [12:40<01:24, 84.89s/it]

Initial portfolio value:100000
Final portfolio value: 1340993.0
Final accumulative portfolio value: 13.40993
Maximum DrawDown: -0.43797168811241527
Sharpe ratio: 1.1121329420590969


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [14:06<00:00, 84.61s/it]
C:\Users\limyi\AppData\Local\Temp\ipykernel_15872\2075129467.py:42: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on G

Initial portfolio value:100000
Final portfolio value: 1374405.625
Final accumulative portfolio value: 13.74405625
Maximum DrawDown: -0.43797137036174094
Sharpe ratio: 1.0983292565708311
Initial portfolio value:100000
Final portfolio value: 102450.5546875
Final accumulative portfolio value: 1.024505546875
Maximum DrawDown: -0.3714851469381545
Sharpe ratio: 0.19266077916396526
An error occurred: [Errno 13] Permission denied: 'results_df.csv'


,TIME_WINDOW,K_SIZE,CONV_MID,CONV_FINAL,LEARNING_RATE,train_portfolio_value,train_drawdown,train_sharpe,train_cumulative,test_portfolio_value,test_drawdown,test_sharpe,test_cumulative
0,100,3,5,5,0.01,2.375308e+06,-0.385159,1.332710,23.753085,112210.781250,-0.297557,0.388886,1.122108
1,100,3,5,5,0.02,3.629857e+05,-0.350651,0.834058,3.629857,117217.054688,-0.159302,0.645506,1.172171
2,100,3,5,10,0.01,9.591604e+05,-0.415274,1.007859,9.591604,112944.757812,-0.314121,0.407323,1.129448
3,100,3,5,10,0.02,3.675439e+05,-0.353164,0.834700,3.675439,117414.390625,-0.161043,0.646087,1.174144
4,100,3,5,20,0.01,1.214352e+06,-0.437965,1.088815,12.143522,90932.937500,-0.371331,-0.048413,0.909329
...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,50,5,20,5,0.02,7.150602e+05,-0.286055,0.875567,7.150602,111502.437500,-0.317670,0.351944,1.115024
158,50,5,20,10,0.01,1.176950e+06,-0.432746,1.021187,11.769497,235457.750000,-0.149673,1.866128,2.354577
159,50,5,20,10,0.02,1.324452e+06,-0.436975,1.081602,13.244525,102449.539062,-0.371485,0.193067,1.024495
160,50,5,20,20,0.01,1.206330e+06,-0.280392,1.062589,12.063301,130736.328125,-0.318687,0.652302,1.307363


In [17]:
results_df

,TIME_WINDOW,K_SIZE,CONV_MID,CONV_FINAL,LEARNING_RATE,train_portfolio_value,train_drawdown,train_sharpe,train_cumulative,test_portfolio_value,test_drawdown,test_sharpe,test_cumulative
0,100,3,5,5,0.01,2.375308e+06,-0.385159,1.332710,23.753085,112210.781250,-0.297557,0.388886,1.122108
1,100,3,5,5,0.02,3.629857e+05,-0.350651,0.834058,3.629857,117217.054688,-0.159302,0.645506,1.172171
2,100,3,5,10,0.01,9.591604e+05,-0.415274,1.007859,9.591604,112944.757812,-0.314121,0.407323,1.129448
3,100,3,5,10,0.02,3.675439e+05,-0.353164,0.834700,3.675439,117414.390625,-0.161043,0.646087,1.174144
4,100,3,5,20,0.01,1.214352e+06,-0.437965,1.088815,12.143522,90932.937500,-0.371331,-0.048413,0.909329
...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,50,5,20,5,0.02,7.150602e+05,-0.286055,0.875567,7.150602,111502.437500,-0.317670,0.351944,1.115024
158,50,5,20,10,0.01,1.176950e+06,-0.432746,1.021187,11.769497,235457.750000,-0.149673,1.866128,2.354577
159,50,5,20,10,0.02,1.324452e+06,-0.436975,1.081602,13.244525,102449.539062,-0.371485,0.193067,1.024495
160,50,5,20,20,0.01,1.206330e+06,-0.280392,1.062589,12.063301,130736.328125,-0.318687,0.652302,1.307363


In [ ]:
results_df2.to_csv('results_df2.csv', index=False)

In [18]:
results_df.to_csv('results_df.csv', index=False)

In [ ]:
FEATURE_NAMES = ['close',
                 'high',
                 'low',
                 'High_VIX',
                 'Low_VIX',
                 'Close_VIX',
                 'High_SPY',
                 'Low_SPY',
                 'Close_SPY',
                 'Volume_SPY',
                 'Fear Greed', 
                 'ratingScore',
                 'ratingDetailsDCFScore',
                 'ratingDetailsROEScore',
                 'ratingDetailsROAScore',
                 'ratingDetailsPEScore',
                 'ratingDetailsPBScore',
                 'expenses',
                 'GDP', 
                 'exports', 
                 'imports',
                 'pound_dollar_exchange_rate',
                 'unemployment',
                 'ics',
                 'TTM Revenue(Billion)',
                 'TTM EBITDA(Billion)',
                 'EBITDA Margin',
                 'Price to Book Ratio',
                 'PE Ratio',
                 'Price to Sales Ratio',
                 'Assets',
                 'NetIncomeLoss',
                 'Return on Equity(%)']

TIME_WINDOW = 50
COMISSION_FEE = 0.0025
K_SIZE = 4
CONV_MID = 5
CONV_FINAL= 20

environment = PortfolioOptimizationEnv(
        df_portfolio_train,
        initial_amount=100000,
        comission_fee_pct=COMISSION_FEE,
        time_window=TIME_WINDOW,
        features=FEATURE_NAMES,
        normalize_df=None,
        reward_function=custom_reward_function,
        reward_scaling=1.0
    )

# set PolicyGradient parameters
model_kwargs = {
    "lr": 0.01,
    "policy": EIIE,
}

# here, we can set EIIE's parameters
policy_kwargs = {
    "initial_features": len(FEATURE_NAMES),
    "k_size": K_SIZE,
    "time_window": TIME_WINDOW,
    "conv_mid_features":CONV_MID,
    "conv_final_features":CONV_FINAL}

EIIE_model = DRLAgent(environment).get_model("pg", device, model_kwargs, policy_kwargs)

# DRL Model Training (Jan 2011- Dec 2020)

In [ ]:
DRLAgent.train_model(EIIE_model, episodes=5)
torch.save(EIIE_model.train_policy.state_dict(), "policy_EIIE(test).pt")

# DRL Model Evaluation (Jan 2021-Dec 2023)

In [ ]:
import os
import shutil

environment_2021 = PortfolioOptimizationEnv(
    df_portfolio_2021,
    initial_amount=100000,
    comission_fee_pct=COMISSION_FEE,
    time_window=TIME_WINDOW,
    features=FEATURE_NAMES,
    normalize_df=None,
    reward_function=custom_reward_function,
    reward_scaling=1.0
)

environment_2022 = PortfolioOptimizationEnv(
    df_portfolio_2022,
    initial_amount=100000,
    comission_fee_pct=COMISSION_FEE,
    time_window=TIME_WINDOW,
    features=FEATURE_NAMES,
    normalize_df=None,
    reward_function=custom_reward_function,
    reward_scaling=1.0
)

environment_2023 = PortfolioOptimizationEnv(
    df_portfolio_2023,
    initial_amount=100000,
    comission_fee_pct=COMISSION_FEE,
    time_window=TIME_WINDOW,
    features=FEATURE_NAMES,
    normalize_df=None,
    reward_function=custom_reward_function,
    reward_scaling=1.0
)

EIIE_results = {
    "train": {},
    "2021": {},
    "2022": {},
    "2023": {}
}
# EI3_results = {
#     "train": {},
#     "2021": {},
#     "2022": {},
#     "2023": {}
# }

# instantiate an architecture with the same arguments used in training
# and load with load_state_dict.

EIIE_policy = EIIE(time_window = TIME_WINDOW, device = device, initial_features = len(FEATURE_NAMES),
              k_size = K_SIZE, conv_mid_features = CONV_MID,conv_final_features = CONV_FINAL)

EIIE_policy.load_state_dict(torch.load("policy_EIIE(test).pt"))

environment.reset()
src_folder = "results/rl"
dst_folder = "results/train"
if not os.path.exists(src_folder):
    os.makedirs(src_folder)
if os.path.exists(dst_folder):
    shutil.rmtree(dst_folder)
DRLAgent.DRL_validation(EIIE_model, environment, policy=EIIE_policy)
os.rename(src_folder, dst_folder)
EIIE_results["train"]["value"] = environment._asset_memory["final"]
EIIE_results["train"]["actions"] = environment._actions_memory

src_folder = "results/rl"
dst_folder = "results/2021"
if not os.path.exists(src_folder):
    os.makedirs(src_folder)
if os.path.exists(dst_folder):
    shutil.rmtree(dst_folder)
DRLAgent.DRL_validation(EIIE_model, environment_2021, policy=EIIE_policy)
os.rename(src_folder, dst_folder)
EIIE_results["2021"]["value"] = environment_2021._asset_memory["final"]
EIIE_results["2021"]["actions"] = environment_2021._actions_memory

src_folder = "results/rl"
dst_folder = "results/2022"
if not os.path.exists(src_folder):
    os.makedirs(src_folder)
if os.path.exists(dst_folder):
    shutil.rmtree(dst_folder)
DRLAgent.DRL_validation(EIIE_model, environment_2022, policy=EIIE_policy)
os.rename(src_folder, dst_folder)
EIIE_results["2022"]["value"] = environment_2022._asset_memory["final"]
EIIE_results["2022"]["actions"] = environment_2022._actions_memory

src_folder = "results/rl"
dst_folder = "results/2023"
if not os.path.exists(src_folder):
    os.makedirs(src_folder)
if os.path.exists(dst_folder):
    shutil.rmtree(dst_folder)
DRLAgent.DRL_validation(EIIE_model, environment_2023, policy=EIIE_policy)
os.rename(src_folder, dst_folder)
EIIE_results["2023"]["value"] = environment_2023._asset_memory["final"]
EIIE_results["2023"]["actions"] = environment_2023._actions_memory


# EI3_policy = EI3(time_window=50, device=device)
# EI3_policy.load_state_dict(torch.load("policy_EI3.pt"))

# environment.reset()
# DRLAgent.DRL_validation(EI3_model, environment, policy=EI3_policy)
# EI3_results["train"]["value"] = environment._asset_memory["final"]
# environment_2021.reset()
# DRLAgent.DRL_validation(EI3_model, environment_2021, policy=EI3_policy)
# EI3_results["2021"]["value"] = environment_2021._asset_memory["final"]
# environment_2022.reset()
# DRLAgent.DRL_validation(EI3_model, environment_2022, policy=EI3_policy)
# EI3_results["2022"]["value"] = environment_2022._asset_memory["final"]
# environment_2023.reset()
# DRLAgent.DRL_validation(EI3_model, environment_2023, policy=EI3_policy)
# EI3_results["2023"]["value"] = environment_2023._asset_memory["final"]

#uniform buy n hold
if not os.path.exists(src_folder):
    os.makedirs(src_folder)
UBAH_results = {
    "train": {"value": [], "actions": []},
    "2021": {"value": [], "actions": []},
    "2022": {"value": [], "actions": []},
    "2023": {"value": [], "actions": []}
}

PORTFOLIO_SIZE = len(TEST_SET)

# train period
terminated = False
environment.reset()
while not terminated:
    action = [0] + [1/PORTFOLIO_SIZE] * PORTFOLIO_SIZE
    _, _, terminated, _ = environment.step(action)
    UBAH_results["train"]["actions"].append(action)
UBAH_results["train"]["value"] = environment._asset_memory["final"]

# 2021
terminated = False
environment_2021.reset()
while not terminated:
    action = [0] + [1/PORTFOLIO_SIZE] * PORTFOLIO_SIZE
    _, _, terminated, _ = environment_2021.step(action)
    UBAH_results["2021"]["actions"].append(action)
UBAH_results["2021"]["value"] = environment_2021._asset_memory["final"]

# 2022
terminated = False
environment_2022.reset()
while not terminated:
    action = [0] + [1/PORTFOLIO_SIZE] * PORTFOLIO_SIZE
    _, _, terminated, _ = environment_2022.step(action)
    UBAH_results["2022"]["actions"].append(action)
UBAH_results["2022"]["value"] = environment_2022._asset_memory["final"]

# 2023
terminated = False
environment_2023.reset()
while not terminated:
    action = [0] + [1/PORTFOLIO_SIZE] * PORTFOLIO_SIZE
    _, _, terminated, _ = environment_2023.step(action)
    UBAH_results["2023"]["actions"].append(action)
UBAH_results["2023"]["value"] = environment_2023._asset_memory["final"]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 

plt.plot(UBAH_results["train"]["value"], label="Buy and Hold")
plt.plot(EIIE_results["train"]["value"], label="EIIE")
#plt.plot(EI3_results["train"]["value"], label="EI3")

plt.xlabel("Days")
plt.ylabel("Portfolio Value")
plt.title("Performance in training period")
plt.legend()

plt.show()

plt.plot(UBAH_results["2021"]["value"], label="Buy and Hold")
plt.plot(EIIE_results["2021"]["value"], label="EIIE")
#plt.plot(EI3_results["2021"]["value"], label="EI3")

plt.xlabel("Days")
plt.ylabel("Portfolio Value")
plt.title("Performance in 2021")
plt.legend()

plt.show()

plt.plot(UBAH_results["2022"]["value"], label="Buy and Hold")
plt.plot(EIIE_results["2022"]["value"], label="EIIE")
#plt.plot(EI3_results["2022"]["value"], label="EI3")

plt.xlabel("Days")
plt.ylabel("Portfolio Value")
plt.title("Performance in 2022")
plt.legend()

plt.show()

plt.plot(UBAH_results["2023"]["value"], label="Buy and Hold")
plt.plot(EIIE_results["2023"]["value"], label="EIIE")
#plt.plot(EI3_results["2023"]["value"], label="EI3")

plt.xlabel("Days")
plt.ylabel("Portfolio Value")
plt.title("Performance in 2023")
plt.legend()

plt.show()

# Portfolio Allocation by Trained DRL Model

In [ ]:
# Function to plot action memory
def plot_action_memory(actions, tickers, label):
    """Visualize portfolio allocation over time."""
    actions_df = pd.DataFrame(actions, columns=["cash"] + tickers)
    actions_df.plot(kind="line", figsize=(12, 8), title=f"Portfolio Allocation Breakdown - {label}")
    plt.ylabel("Portfolio Weight")
    plt.xlabel("Time Step")
    plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Plot EIIE actions for the training period
plot_action_memory(EIIE_results["train"]["actions"], TEST_SET, "EIIE - Train")

# Plot EIIE actions for 2021
plot_action_memory(EIIE_results["2021"]["actions"], TEST_SET, "EIIE - 2021")

# Plot EIIE actions for 2022
plot_action_memory(EIIE_results["2022"]["actions"], TEST_SET, "EIIE - 2022")

# Plot EIIE actions for 2023
plot_action_memory(EIIE_results["2023"]["actions"], TEST_SET, "EIIE - 2023")

# Similarly, plot UBAH actions for each period
# plot_action_memory(UBAH_results["train"]["actions"], TEST_SET, "UBAH - Train")
# plot_action_memory(UBAH_results["2021"]["actions"], TEST_SET, "UBAH - 2021")
# plot_action_memory(UBAH_results["2022"]["actions"], TEST_SET, "UBAH - 2022")
# plot_action_memory(UBAH_results["2023"]["actions"], TEST_SET, "UBAH - 2023")

In [ ]:
results_df2 = results_df2[["TIME_WINDOW", "K_SIZE", "CONV_MID", "CONV_FINAL", 
            'train_portfolio_value', 'train_drawdown', 'train_sharpe','train_cumulative',
            '2021_portfolio_value', '2021_drawdown', '2021_sharpe','2021_cumulative',
             '2022_portfolio_value', '2022_drawdown', '2022_sharpe','2022_cumulative',
             '2023_portfolio_value', '2023_drawdown', '2023_sharpe','2023_cumulative']]